# Análise Fundiária – Frutal-MG

Este notebook realiza uma análise fundiária do município de Frutal-MG, utilizando dados públicos de imóveis rurais. A seguir, são aplicadas técnicas de limpeza, estatística e visualização para revelar padrões de concentração de terra.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Leitura do Dataset

In [ ]:
df = pd.read_csv("data/Imoveis_3127107.csv", sep=";", encoding="latin1")

## Renomear Colunas e Tratar Área

In [ ]:
df.columns = [
    'codigo_imovel', 'denominacao_imovel', 'codigo_municipio',
    'municipio', 'uf', 'area_total', 'titular',
    'natureza_juridica', 'condicao_pessoa', 'percentual_detencao', 'pais'
]

df['area_total'] = df['area_total'].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df['area_total'] = pd.to_numeric(df['area_total'], errors='coerce')

## Normalização de Natureza Jurídica

In [ ]:
def limpar_texto(texto):
    try:
        return texto.encode('latin1').decode('utf-8').upper().strip()
    except:
        return str(texto).upper().strip()

df['natureza_juridica_normalizada'] = df['natureza_juridica'].apply(limpar_texto)

In [ ]:
df_unicos = df.drop_duplicates(subset='codigo_imovel').copy()
df_unicos['primeiro_nome'] = df_unicos['titular'].astype(str).str.strip().str.upper().str.split().str[0]

## Índice de Gini Fundiário

In [ ]:
def gini(array):
    array = np.sort(array[~np.isnan(array)])
    n = array.size
    index = np.arange(1, n + 1)
    return ((np.sum((2 * index - n - 1) * array)) / (n * np.sum(array))

gini_index = gini(df_unicos['area_total'].values)
print(f"Gini Fundiário: {gini_index:.4f}")

## Estatísticas por Natureza Jurídica

In [ ]:
df_unicos.groupby('natureza_juridica_normalizada')['area_total'].agg(['count', 'mean', 'median', 'std']).sort_values(by='mean', ascending=False)

## Visualizações – Histograma e Boxplot

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df_unicos['area_total'], bins=50, kde=True, color='steelblue')
plt.title('Distribuição do Tamanho das Propriedades Rurais')
plt.xlabel('Área Total (ha)')
plt.ylabel('Quantidade de Imóveis')
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(14, 6))
sns.boxplot(data=df_unicos, x='natureza_juridica_normalizada', y='area_total')
plt.xticks(rotation=90)
plt.title('Boxplot de Área por Natureza Jurídica')
plt.ylabel('Área (ha)')
plt.xlabel('Natureza Jurídica')
plt.tight_layout()
plt.show()